# 🚀 Setup
Runs with Pyton 3.8.2 64 bit base conda kernel



Once you've picked a `method` to try out new values of the hyperparameters,
you need to define what those `parameters` are.

Most of the time, this step is straightforward:
you just give the `parameter` a name
and specify a list of legal `values`
of the parameter.

For example, when we choose the `optimizer` for our network,
there's only a finite number of options.
Here we stick with the two most popular choices, `adam` and `sgd`.
Even for hyperparameters that have potentially infinite options,
it usually only makes sense to try out
a few select `values`,
as we do here with the hidden `layer_size` and `dropout`.

In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train(config=None):
    # Initialize a new wandb run


    loader = build_dataset(64)
    network = build_network()
    optimizer = build_optimizer(network, "adam", 1e-3)

    for epoch in range(100):
        print(" hello train")
        print(epoch)
        error = train_epoch(network, loader, optimizer)
        print(" hello poch ende")

        print(error)
    print("end of loop")        

# Build data loader

In [4]:
import pyarrow.feather as feather
import numpy as np




This cell defines the four pieces of our training procedure:
`build_dataset`, `build_network`, `build_optimizer`, and `train_epoch`.

All of these are a standard part of a basic PyTorch pipeline,
and their implementation is unaffected by the use of W&B,
so we won't comment on them.

In [5]:
def build_dataset(batch_size):
    fluxData_df = feather.read_feather('data/fluxData.feather')
    zernikeData_df = feather.read_feather('data/zernikeData.feather')


    train_target = torch.tensor(zernikeData_df.values.astype(np.float32))
    train = torch.tensor(fluxData_df.values.astype(np.float32))

    train_tensor = torch.utils.data.TensorDataset(train, train_target) 
    loader = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)
   


    return loader

def build_network():
    network = nn.Sequential( 
    nn.Linear(19,2000), nn.ReLU(),
    nn.Linear(2000,1050), nn.ReLU(),
    nn.Linear(1050,100), nn.ReLU(),
    nn.Linear(100, 9))

    return network.to(device)
        

def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer


def train_epoch(network, loader, optimizer):
    cumu_loss = 0
    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
       # print(" hello train_epoch")


        # ➡ Forward pass
        loss = nn.MSELoss()
        loss =loss(network(data), target)
        #cumu_loss += loss
        lossRMS= nn.MSELoss()
        RmsLossValue=torch.sqrt(lossRMS(torch.flatten(network(data)), torch.flatten(target)))

        # ⬅ Backward pass + weight update
        loss.backward()
        optimizer.step()

        #wandb.log({"batch loss": RmsLossValue, "RMS Loss" : RmsLossValue})

    return RmsLossValue

In [6]:
train()

 hello train
0
 hello poch ende
tensor(0.0516, device='cuda:0', grad_fn=<SqrtBackward>)
 hello train
1
 hello poch ende
tensor(0.0524, device='cuda:0', grad_fn=<SqrtBackward>)
 hello train
2
 hello poch ende
tensor(0.0434, device='cuda:0', grad_fn=<SqrtBackward>)
 hello train
3
 hello poch ende
tensor(0.0346, device='cuda:0', grad_fn=<SqrtBackward>)
 hello train
4
 hello poch ende
tensor(0.0302, device='cuda:0', grad_fn=<SqrtBackward>)
 hello train
5
 hello poch ende
tensor(0.0321, device='cuda:0', grad_fn=<SqrtBackward>)
 hello train
6
 hello poch ende
tensor(0.0268, device='cuda:0', grad_fn=<SqrtBackward>)
 hello train
7
 hello poch ende
tensor(0.0272, device='cuda:0', grad_fn=<SqrtBackward>)
 hello train
8
 hello poch ende
tensor(0.0291, device='cuda:0', grad_fn=<SqrtBackward>)
 hello train
9
 hello poch ende
tensor(0.0266, device='cuda:0', grad_fn=<SqrtBackward>)
 hello train
10
 hello poch ende
tensor(0.0256, device='cuda:0', grad_fn=<SqrtBackward>)
 hello train
11
 hello poch end

In [ ]:
# a = torch.flatten(torch.subtract(network(data),target))
# lossRMS= nn.MSELoss()
# a=torch.sqrt(lossRMS(torch.flatten(network(data)), torch.flatten(target)))
